<a href="https://colab.research.google.com/github/Ron-po/tracking-swimmers/blob/main/tracking_swimmers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Start here

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from collections import Counter
import imageio
import os

# Get Red Lines Mask

def largest_connected_component(mask):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
    if num_labels <= 1:
        return mask
    largest_label = 1
    largest_area = stats[1, cv2.CC_STAT_AREA]
    for i in range(2, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area > largest_area:
            largest_area = area
            largest_label = i
    largest_mask = np.zeros_like(mask, dtype=np.uint8)
    largest_mask[labels == largest_label] = 255
    return largest_mask

def detect_pool(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([40, 50, 50])
    upper_blue = np.array([130, 240, 220])
    pool_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    pool_mask = largest_connected_component(pool_mask)

    # Find contours around the pool
    contours, _ = cv2.findContours(pool_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        print("No pool found!")
        return None

    largest_contour = max(contours, key=cv2.contourArea)

    # Draw the largest contour (assuming it's the pool)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)  # Get the largest contour

    # Create an empty mask
    filled_mask = np.zeros_like(pool_mask)

    # Fill in the detected pool area
    cv2.drawContours(filled_mask, [largest_contour], -1, 255, thickness=cv2.FILLED)

    return filled_mask  # Return the filled mask instead of the contour

def remove_blues_and_blacks(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define range for blue and black in HSV space
    lower_blue = np.array([90, 50, 50])  # Lower bound for blue
    upper_blue = np.array([150, 255, 255])  # Upper bound for blue

    # Define range for black (low saturation and value) in HSV space
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])  # Very dark colors

    # Create masks for blue and black
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    black_mask = cv2.inRange(hsv, lower_black, upper_black)

    # Combine the two masks (blue and black)
    combined_mask = cv2.bitwise_or(blue_mask, black_mask)

    # Invert the mask to keep everything except blue and black
    inverted_mask = cv2.bitwise_not(combined_mask)

    return inverted_mask

def detect_lane_lines(image):
    # Convert to grayscale (needed for edge detection)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection to find edges
    edges = cv2.Canny(blurred, 50, 150)

    # Detect lines using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=50)

    # Draw detected lines in red
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)  # red lines

    return image, lines

def theta_from_slope(slope):
    return np.degrees(np.arctan(slope))

def remove_outliers(merged_lines):
    """
    Remove outliers from a list of nearly parallel lines based on their slopes.
    Keep the set of slopes that is most common.
    merged_lines: NumPy array of shape (n, 4), where each row represents a line (x1, y1, x2, y2)
    """
    # Calculate angles for each line
    lines = merged_lines[:, 0]
    thetas = [theta_from_slope(line) for line in lines]

    # Find the median angle
    median_angle = np.median(thetas)

    # Set a threshold for outlier removal (based on a small deviation from the most common angle)
    threshold = 5  # Adjust threshold as needed

    filtered_lines = []
    for i, line in enumerate(merged_lines[:, 0]):
        angle_deg = theta_from_slope(line)
        if np.abs(angle_deg - median_angle) <= threshold:
            filtered_lines.append(merged_lines[i])

    return np.array(filtered_lines)

def merge_lane_lines(lines, max_height=10000, slope_threshold=0.2, intercept_threshold=20):
    if lines is None:
        return []

    merged_lines = []

    # Group lines with similar slopes and intercepts
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if x2 - x1 != 0:  # Prevent division by zero
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - slope * x1  # y = mx + b -> b = y - mx

            if intercept < 0 or intercept > max_height:
                continue

            merged = False
            for idx, (m, b) in enumerate(merged_lines):
                if abs(slope - m) < slope_threshold and abs(intercept - b) < intercept_threshold:
                    merged_lines[idx] = ((m + slope) / 2, (b + intercept) / 2)
                    merged = True
                    break

            if not merged:
                merged_lines.append((slope, intercept))

    merged_lines = np.array(merged_lines)
    merged_lines = remove_outliers(merged_lines)
    merged_lines = merged_lines[merged_lines[:, 1].argsort()]

    return merged_lines

def draw_merged_lines(image, merged_lines):
    # Draw merged lane lines in red
    for slope, intercept in merged_lines:
        x1 = 0
        x2 = int(image.shape[1])
        y1 = int(intercept)
        y2 = int(slope * x2 + intercept)
        cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)  # red line

    return image

def rotate_image(image, theta):
    (height, width) = image.shape[:2]
    center = (width // 2, height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, theta, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height), flags=cv2.INTER_LINEAR)
    return rotated_image

# Modified crop_lane: cropping is removed so the full image is returned.
def crop_lane(image, lane, merged_lines):
    # Simply return the original image (no cropping performed)
    return image, image

rotation_angles = []

def process(frame, lane, old_merged_lines):
    pool_mask = detect_pool(frame)
    pool = cv2.bitwise_and(frame, frame, mask=pool_mask)
    blue_black_mask = remove_blues_and_blacks(pool)
    pool_no_blue = cv2.bitwise_and(pool, pool, mask=blue_black_mask)
    result_image, lines = detect_lane_lines(pool_no_blue)
    new_merged_lines = merge_lane_lines(lines, frame.shape[0])
    if len(new_merged_lines) >= lane:
        old_merged_lines = new_merged_lines

    # Create a black image
    blacked_out_image = np.zeros_like(frame)

    # Draw only the final kept red lines on the black background
    for slope, intercept in old_merged_lines:
        x1 = 0
        x2 = int(frame.shape[1])
        y1 = int(intercept)
        y2 = int(slope * x2 + intercept)
        cv2.line(blacked_out_image, (x1, y1), (x2, y2), (0, 0, 255), 4)  # Red lines on black background

    avg_slope = (old_merged_lines[lane - 2, 0] + old_merged_lines[lane - 1, 0]) / 2
    theta = theta_from_slope(avg_slope)

    rotation_angles.append(theta)

    # Rotate the blacked-out image with red lines
    rotated_image = rotate_image(blacked_out_image, theta)

    # Apply rotation to the final output by using the rotated image in the crop_lane function.
    cropped_image, masked_lane = crop_lane(rotated_image, lane, old_merged_lines)

    return old_merged_lines, cropped_image, masked_lane



# Main execution to process a video and save the output
video_path = "200free.mp4"  # Ensure the video is in the same folder
lane = 3

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video '{video_path}'. Check if it's uploaded.")
else:
    ret, frame = cap.read()
    if not ret:
        print("Error: No frames in the video.")
        cap.release()
    else:
        old_merged_lines = np.array([])
        old_merged_lines, cropped_image, masked_image = process(frame, lane, old_merged_lines)

        height, width = masked_image.shape[:2]
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        output_path = "red_lines_mask.mp4"
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

        out.write(masked_image)

        frame_num = 1

        while True:
            ret, frame = cap.read()
            if not ret:
                print("End of video or cannot read frame.")
                break
            old_merged_lines, cropped_image, masked_image = process(frame, lane, old_merged_lines)
            out.write(masked_image)


            frame_num += 1

        cap.release()
        out.release()
        print("\nDone processing video. Saved as:", output_path)


End of video or cannot read frame.

Done processing video. Saved as: red_lines_mask.mp4


In [ ]:
# Get only swimmers and lanes - while matching dimensions to red_lines_mask

import cv2
import numpy as np

# Video Paths
video_path = "200free.mp4"
output_path = "tracked_swimmers.mp4"  # Color video with non-water overlay
difference_output_path = "non_water_mask.mp4"  # Black-and-white difference video

# Open the video and get properties
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"Error: Could not open video {video_path}.")
    exit()

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Define HSV thresholds for detecting pool water (blue color)
lower_blue = np.array([40, 50, 50])
upper_blue = np.array([130, 255, 255])

# Initialize video writers (keeping original resolution)
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
out_diff = cv2.VideoWriter(difference_output_path, fourcc, fps, (frame_width, frame_height))

# ========================
# Process Each Frame
# ========================
while True:
    ret, frame = cap.read()
    if not ret:
        break  # End of video

    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    pool_mask = cv2.inRange(hsv, lower_blue, upper_blue)

    # Find contours in the pool mask and select the largest (assumed to be the pool area)
    contours, _ = cv2.findContours(pool_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        connected_mask = np.zeros_like(pool_mask)
        cv2.drawContours(connected_mask, [largest_contour], -1, 255, thickness=-1)

        # Compute non-water regions: subtract pool mask from connected mask
        non_water_mask = cv2.subtract(connected_mask, pool_mask)
    else:
        # If no pool detected, assume entire frame is non-water
        non_water_mask = np.zeros_like(pool_mask)

    # ===========================
    # Generate Overlay Output
    # ===========================
    water_region = cv2.bitwise_and(frame, frame, mask=pool_mask)
    final_frame = water_region.copy()
    final_frame[non_water_mask == 255] = (0, 255, 0)  # Highlight non-water in green

    # Write full-resolution frame with green overlay
    out.write(final_frame)

    # ===========================
    # Generate Black-and-White Mask Output
    # ===========================
    diff_image = np.zeros_like(frame)  # Black background
    diff_image[non_water_mask == 255] = (255, 255, 255)  # White for non-water regions

    # Write full-resolution difference mask
    out_diff.write(diff_image)

cap.release()
out.release()
out_diff.release()

print(f"Processing complete.\n"
      f" - Full color video with green overlay saved as: {output_path}\n"
      f" - Black-and-white difference video saved as: {difference_output_path}")


Processing complete.
 - Full color video with green overlay saved as: tracked_swimmers.mp4
 - Black-and-white difference video saved as: non_water_mask.mp4


In [ ]:
# Rotate non_water_mask to align with red_lines_mask


# rotation angles obtained from previous script
rotation_angles

# Open non_water_mask video
non_water_video_path = "non_water_mask.mp4"
cap_non_water = cv2.VideoCapture(non_water_video_path)

if not cap_non_water.isOpened():
    print(f"Error: Could not open video '{non_water_video_path}'.")
else:
    # Get video properties
    fps = int(cap_non_water.get(cv2.CAP_PROP_FPS))
    width = int(cap_non_water.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_non_water.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = "non_water_mask_rotated.mp4"
    out_non_water = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_index = 0
    while True:
        ret, frame = cap_non_water.read()
        if not ret:
            print("End of non_water_mask video or cannot read frame.")
            break

        if frame_index < len(rotation_angles):
            theta = rotation_angles[frame_index]  # Correct index usage
        else:
            print(f"Warning: Missing rotation angle for frame {frame_index}. Using last available angle.")
            theta = rotation_angles[-1]

        # Rotate the frame by the corresponding angle
        (h, w) = frame.shape[:2]
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, theta, 1.0)
        rotated_frame = cv2.warpAffine(frame, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)

        out_non_water.write(rotated_frame)
        frame_index += 1

    # Release everything
    cap_non_water.release()
    out_non_water.release()

    print(f"Rotated non_water_mask video saved as {output_path}")


End of non_water_mask video or cannot read frame.
Rotated non_water_mask video saved as non_water_mask_rotated.mp4


In [ ]:
# Rotate non_water_mask to align with red_lines_mask


# rotation angles obtained from previous script
rotation_angles

# Open non_water_mask video
non_water_video_path = "200free.mp4"
cap_non_water = cv2.VideoCapture(non_water_video_path)

if not cap_non_water.isOpened():
    print(f"Error: Could not open video '{non_water_video_path}'.")
else:
    # Get video properties
    fps = int(cap_non_water.get(cv2.CAP_PROP_FPS))
    width = int(cap_non_water.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_non_water.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_path = "200free_rotated.mp4"
    out_non_water = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    frame_index = 0
    while True:
        ret, frame = cap_non_water.read()
        if not ret:
            print("End of non_water_mask video or cannot read frame.")
            break

        if frame_index < len(rotation_angles):
            theta = rotation_angles[frame_index]  # Correct index usage
        else:
            print(f"Warning: Missing rotation angle for frame {frame_index}. Using last available angle.")
            theta = rotation_angles[-1]

        # Rotate the frame by the corresponding angle
        (h, w) = frame.shape[:2]
        center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, theta, 1.0)
        rotated_frame = cv2.warpAffine(frame, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)

        out_non_water.write(rotated_frame)
        frame_index += 1

    # Release everything
    cap_non_water.release()
    out_non_water.release()

    print(f"Rotated non_water_mask video saved as {output_path}")


End of non_water_mask video or cannot read frame.
Rotated non_water_mask video saved as 200free_rotated.mp4


In [ ]:
# Track swimmers

red_lines_video = "red_lines_mask.mp4"
non_water_video = "non_water_mask_rotated.mp4"
output_black = "final_overlay_black.mp4"
output_red = "final_overlay_red.mp4"

# Open both videos
cap_red = cv2.VideoCapture(red_lines_video)
cap_non_water = cv2.VideoCapture(non_water_video)

# Check if videos opened successfully
if not cap_red.isOpened() or not cap_non_water.isOpened():
    print("Error: Could not open one or both videos.")
    exit()

# Get video properties (assuming both have the same properties)
fps = int(cap_red.get(cv2.CAP_PROP_FPS))
frame_width = int(cap_red.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_red.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Initialize video writers
out_black = cv2.VideoWriter(output_black, fourcc, fps, (frame_width, frame_height))
out_red = cv2.VideoWriter(output_red, fourcc, fps, (frame_width, frame_height))

# Define red color range in HSV (for better detection)
lower_hsv_red = np.array([0, 120, 100])   # Lower bound for red
upper_hsv_red = np.array([10, 255, 255])    # Upper bound for red

# ------------------------------------------------------------------
# AUTOMATIC RED LANE DETECTION & STORAGE (using the first frame)
# ------------------------------------------------------------------
# Read the first frame from the red-lines video
ret_first, first_frame_red = cap_red.read()
if not ret_first:
    print("Error: Could not read first frame from red_lines_mask.mp4")
    exit()

# Convert to HSV and create a red mask
hsv_first = cv2.cvtColor(first_frame_red, cv2.COLOR_BGR2HSV)
red_mask_first = cv2.inRange(hsv_first, lower_hsv_red, upper_hsv_red)

# Thicken the red lines (same as below)
kernel = np.ones((5, 5), np.uint8)
thickened_mask_first = cv2.dilate(red_mask_first, kernel, iterations=2)
# Further thicken the bottom half of the frame
bottom_part_first = thickened_mask_first[frame_height // 2:]
extra_thick_kernel = np.ones((7, 7), np.uint8)
bottom_part_thicker_first = cv2.dilate(bottom_part_first, extra_thick_kernel, iterations=4)
thickened_mask_first[frame_height // 2:] = bottom_part_thicker_first

# Automatically detect red lines by scanning rows
min_line_pixel_count = 50  # Adjust threshold as needed
line_rows = []
for row in range(frame_height):
    if np.sum(thickened_mask_first[row, :] == 255) > min_line_pixel_count:
        line_rows.append(row)

# Group consecutive rows into single lines
line_groups = []
if line_rows:
    current_group = [line_rows[0]]
    for i in range(1, len(line_rows)):
        if line_rows[i] == line_rows[i-1] + 1:
            current_group.append(line_rows[i])
        else:
            line_groups.append(current_group)
            current_group = [line_rows[i]]
    if current_group:
        line_groups.append(current_group)
else:
    print("Warning: No red lines detected.")

# Compute average row for each detected red line
red_line_positions = [int(sum(group)/len(group)) for group in line_groups]
red_line_positions.sort()

# Only consider the topmost 6 red lines (from top of image to bottom)
if len(red_line_positions) >= 6:
    lane_line_positions = red_line_positions[:6]
else:
    lane_line_positions = red_line_positions

# Compute lane boundaries (5 lanes between 6 lines)
lane_bounds = []
for i in range(len(lane_line_positions) - 1):
    lane_bounds.append((lane_line_positions[i], lane_line_positions[i+1]))
print("Detected lane boundaries (y_top, y_bottom):", lane_bounds)

# Reset red-lines video to beginning (if needed)
cap_red.set(cv2.CAP_PROP_POS_FRAMES, 0)

# ========================
# Process Each Frame for Final Overlays
# ========================
while True:
    ret_red, frame_red = cap_red.read()
    ret_non_water, frame_non_water = cap_non_water.read()

    if not ret_red or not ret_non_water:
        break  # Stop if either video ends

    # Convert red_lines_mask frame to HSV for better red detection
    hsv_red = cv2.cvtColor(frame_red, cv2.COLOR_BGR2HSV)

    # Create mask for red pixels
    red_mask = cv2.inRange(hsv_red, lower_hsv_red, upper_hsv_red)

    # **Thicken all lane lines slightly**
    kernel = np.ones((5, 5), np.uint8)
    thickened_mask = cv2.dilate(red_mask, kernel, iterations=2)

    # **Identify Bottom 3 Lane Lines & Thicken Them More**
    bottom_part = thickened_mask[frame_height // 2:]
    extra_thick_kernel = np.ones((7, 7), np.uint8)
    bottom_part_thicker = cv2.dilate(bottom_part, extra_thick_kernel, iterations=4)

    # **Merge top and thickened bottom**
    thickened_mask[frame_height // 2:] = bottom_part_thicker

    # Overwrite pixels: create final_frame_black by drawing black over detected red regions.
    final_frame_black = frame_non_water.copy()
    contours, _ = cv2.findContours(thickened_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(final_frame_black, contours, -1, (0, 0, 0), thickness=cv2.FILLED)

    # Similarly, create final_frame_red by drawing red over detected regions.
    final_frame_red = frame_non_water.copy()
    cv2.drawContours(final_frame_red, contours, -1, (0, 0, 255), thickness=cv2.FILLED)

    # Write frames to both output videos
    out_black.write(final_frame_black)
    out_red.write(final_frame_red)

# Release everything
cap_red.release()
cap_non_water.release()
out_black.release()
out_red.release()

print(f"Processing complete. Videos saved as:\n"
      f" - Blacked-out version: {output_black}\n"
      f" - Red overlay version: {output_red}")


Detected lane boundaries (y_top, y_bottom): [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Processing complete. Videos saved as:
 - Blacked-out version: final_overlay_black.mp4
 - Red overlay version: final_overlay_red.mp4


In [ ]:
#Black out irrelevant regions

from moviepy.editor import VideoFileClip, VideoClip, CompositeVideoClip
import numpy as np

# Load the original video
clip = VideoFileClip("final_overlay_red.mp4")
w, h = clip.size
fps = clip.fps

# Define the time (in seconds) at which to start expanding the blackout region.
T0 = 380 / fps  # frame 350

# Define the initial and final blackout heights.
initial_height = int(0.35 * h)   # initial height to black out (from bottom)
final_height = int(0.45 * h)            # final blackout height after transition

# Define the duration over which the transition will occur (in seconds)
T_transition = 2.0  # adjust as needed for a smoother transition

# Function to generate the black rectangle frame dynamically.
def black_rect_frame(t):
    if t < T0:
        current_height = initial_height
    elif t < T0 + T_transition:
        progress = (t - T0) / T_transition
        current_height = int((1 - progress) * initial_height + progress * final_height)
    else:
        current_height = final_height
    # Create a black image with the computed height.
    return np.zeros((current_height, w, 3), dtype=np.uint8)

# Function to position the black rectangle so that its bottom aligns with the video bottom.
def black_pos(t):
    if t < T0:
        current_height = initial_height
    elif t < T0 + T_transition:
        progress = (t - T0) / T_transition
        current_height = int((1 - progress) * initial_height + progress * final_height)
    else:
        current_height = final_height
    # Position it at the bottom: x=0, y = video height - current_height.
    return (0, h - current_height)

# Create a dynamic black clip that lasts the duration of the original clip.
black_clip = VideoClip(black_rect_frame, duration=clip.duration)
black_clip = black_clip.set_pos(black_pos)

# Overlay the dynamic black clip on top of the original clip.
final_clip = CompositeVideoClip([clip, black_clip])

# Write the result to a new video file.
final_clip.write_videofile("relevant_vid.mp4", codec="libx264", audio_codec="aac")


  if event.key is 'enter':



Moviepy - Building video relevant_vid.mp4.
Moviepy - Writing video relevant_vid.mp4



Moviepy - Done !
Moviepy - video ready relevant_vid.mp4


In [ ]:
# import cv2
# import numpy as np

# # Ensure that lane_bounds (a list of 5 tuples) is available from Cell 1.
# print("Using lane bounds:", lane_bounds)

# # Input video path for final overlay red video
# overlay_red_video = "relevant_vid.mp4"
# cap_swimmers = cv2.VideoCapture(overlay_red_video)
# if not cap_swimmers.isOpened():
#     print("Error: Could not open", overlay_red_video)
#     exit()

# fps = int(cap_swimmers.get(cv2.CAP_PROP_FPS))
# frame_width = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out_swimmers = cv2.VideoWriter("tracked_swimmers.mp4", fourcc, fps, (frame_width, frame_height))

# frame_index = 0
# # Dictionary to store current centroid for each lane (keys: 0 to 4)
# centroids = {}
# # Dictionary to store previous centroids (for speed computation)
# prev_centroids = {}

# # Conversion factor: 61.8 pixels per yard
# pixels_per_yard = 61.8

# while True:
#     ret, frame = cap_swimmers.read()
#     if not ret:
#         break

#     # Start tracking only after frame 100; before that, write unmodified frame.
#     if frame_index < 100:
#         out_swimmers.write(frame)
#         frame_index += 1
#         continue

#     # Convert frame to grayscale and threshold to isolate white blobs (swimmers)
#     gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     # Adjust threshold if needed so that swimmers (white) remain bright.
#     _, swimmer_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

#     # Process each of the 5 lanes (from lane_bounds)
#     for lane_idx, (y_top, y_bottom) in enumerate(lane_bounds):
#         if lane_idx >= 5:
#             break  # only process 5 lanes

#         # Extract the ROI corresponding to the lane (between the two red lines)
#         lane_roi = swimmer_mask[y_top:y_bottom, :]
#         contours, _ = cv2.findContours(lane_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#         if contours:
#             # Select the largest contour (assumed to be the swimmer)
#             largest_contour = max(contours, key=cv2.contourArea)
#             if cv2.contourArea(largest_contour) > 30:  # Filter out noise
#                 M = cv2.moments(largest_contour)
#                 if M["m00"] != 0:
#                     cx = int(M["m10"] / M["m00"])
#                     cy = int(M["m01"] / M["m00"]) + y_top  # adjust y-coordinate to full-frame
#                     centroids[lane_idx] = (cx, cy)
#         # If no blob is detected and it's the first tracking frame, initialize centroid on the left.
#         if lane_idx not in centroids and frame_index == 100:
#             init_x = int(0.1 * frame_width)  # initialize on the left (10% of frame width)
#             init_y = int((y_top + y_bottom) / 2)
#             centroids[lane_idx] = (init_x, init_y)

#     # Compute speed for each lane (in yards per second) if previous centroid is available
#     speeds = {}  # speed per lane in yd/s
#     for lane_idx in range(5):
#         if lane_idx in centroids and lane_idx in prev_centroids:
#             (prev_x, prev_y) = prev_centroids[lane_idx]
#             (curr_x, curr_y) = centroids[lane_idx]
#             distance = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2)
#             # Convert pixels per second to yards per second
#             speeds[lane_idx] = (distance * fps) / pixels_per_yard
#         else:
#             speeds[lane_idx] = 0.0

#     # Draw centroids, swimmer labels, and speed on the frame
#     for lane_idx in range(5):
#         if lane_idx in centroids:
#             cx, cy = centroids[lane_idx]
#             cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
#             # Display swimmer label
#             cv2.putText(frame, f"Swimmer {lane_idx+1}", (cx + 10, cy),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
#             # Display speed above the swimmer (20 pixels above the centroid)
#             speed_text = f"{speeds[lane_idx]:.1f} yd/s"
#             cv2.putText(frame, speed_text, (cx + 10, cy - 20),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

#     # Update previous centroids for next frame
#     prev_centroids = centroids.copy()

#     out_swimmers.write(frame)
#     frame_index += 1

# cap_swimmers.release()
# out_swimmers.release()
# print("Swimmer tracking complete. Output saved as 'tracked_swimmers.mp4'")


Using lane bounds: [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Swimmer tracking complete. Output saved as 'tracked_swimmers.mp4'


In [ ]:
# Only centroids, no speeds
import cv2
import numpy as np

print("Using lane bounds:", lane_bounds)

overlay_red_video = "relevant_vid.mp4"
cap_swimmers = cv2.VideoCapture(overlay_red_video)
if not cap_swimmers.isOpened():
    print("Error: Could not open", overlay_red_video)
    exit()

fps = int(cap_swimmers.get(cv2.CAP_PROP_FPS))
frame_width = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_swimmers = cv2.VideoWriter("only_centroids.mp4", fourcc, fps, (frame_width, frame_height))

frame_index = 0
# Threshold frame: after 7 seconds, stop updating lane 5 (index 4) and shift labels.
threshold_frame = int(7 * fps)

# Dictionary to store current centroids (lane_idx -> (cx, cy))
centroids = {}
prev_centroids = {}

while True:
    ret, frame = cap_swimmers.read()
    if not ret:
        break

    # For the first 100 frames, write unmodified frame.
    if frame_index < 100:
        out_swimmers.write(frame)
        frame_index += 1
        continue

    # Convert frame to grayscale and threshold to isolate white blobs (swimmers)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, swimmer_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # Process each lane (using lane_bounds)
    for lane_idx, (y_top, y_bottom) in enumerate(lane_bounds):
        if lane_idx >= 5:
            break  # only process 5 lanes

        # After 7 seconds, do not update lane 5 (index 4)
        if frame_index >= threshold_frame and lane_idx == 4:
            continue

        # Extract the ROI corresponding to the lane (between the two red lines)
        lane_roi = swimmer_mask[y_top:y_bottom, :]
        contours, _ = cv2.findContours(lane_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        new_centroid = None

        if contours:
            # Select the largest contour (assumed to be the swimmer)
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:  # Filter out noise
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"]) + y_top  # adjust y-coordinate to full-frame
                    new_centroid = (cx, cy)

        # If no detection and it's the first tracking frame, initialize on the left.
        if new_centroid is None and lane_idx not in centroids and frame_index == 100:
            init_x = int(0.1 * frame_width)
            init_y = int((y_top + y_bottom) / 2)
            centroids[lane_idx] = (init_x, init_y)
        elif new_centroid is not None:
            centroids[lane_idx] = new_centroid

    # Draw centroids and labels on the frame.
    # For frames after 7 seconds, shift the labels by +1 and do not display lane 5.
    for lane_idx in range(5):
        if frame_index >= threshold_frame and lane_idx == 4:
            continue
        if lane_idx in centroids:
            cx, cy = centroids[lane_idx]
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
            if frame_index >= threshold_frame:
                label = f"Swimmer {lane_idx+1+1}"  # shift label by +1
            else:
                label = f"Swimmer {lane_idx+1}"
            cv2.putText(frame, label, (cx + 10, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    prev_centroids = centroids.copy()
    out_swimmers.write(frame)
    frame_index += 1

cap_swimmers.release()
out_swimmers.release()
print("Swimmer tracking complete. Output saved as 'only_centroids.mp4'")


Using lane bounds: [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Swimmer tracking complete. Output saved as 'only_centroids.mp4'


In [ ]:
import cv2
import numpy as np

# Remove bad labeling

print("Using lane bounds:", lane_bounds)

overlay_red_video = "relevant_vid.mp4"
cap_swimmers = cv2.VideoCapture(overlay_red_video)
if not cap_swimmers.isOpened():
    print("Error: Could not open", overlay_red_video)
    exit()

fps = int(cap_swimmers.get(cv2.CAP_PROP_FPS))
frame_width = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_swimmers.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_swimmers = cv2.VideoWriter("tracked_swimmers.mp4", fourcc, fps, (frame_width, frame_height))

frame_index = 0
# Threshold frame: after 7 seconds, stop updating lane index 4
threshold_frame = int(7 * fps)

# Dictionaries to store current centroids (lane_idx -> (cx, cy))
# and previous centroids (for speed computation)
centroids = {}
prev_centroids = {}

# Conversion factor: 61.8 pixels per yard
pixels_per_yard = 61.8

while True:
    ret, frame = cap_swimmers.read()
    if not ret:
        break

    # For the first 100 frames, write unmodified frame.
    if frame_index < 100:
        out_swimmers.write(frame)
        frame_index += 1
        continue

    # Convert frame to grayscale and threshold to isolate white blobs (swimmers)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    _, swimmer_mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)

    # Process each lane (from lane_bounds)
    for lane_idx, (y_top, y_bottom) in enumerate(lane_bounds):
        if lane_idx >= 5:
            break  # only process 5 lanes

        # After 7 seconds, do not update lane 5 (index 4)
        if frame_index >= threshold_frame and lane_idx == 4:
            continue

        # Extract the ROI corresponding to the lane (between the two red lines)
        lane_roi = swimmer_mask[y_top:y_bottom, :]
        contours, _ = cv2.findContours(lane_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        new_centroid = None

        if contours:
            # Select the largest contour (assumed to be the swimmer)
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:  # Filter out noise
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"]) + y_top  # adjust y-coordinate to full-frame
                    new_centroid = (cx, cy)

        # If no detection and it's the first tracking frame, initialize on the left.
        if new_centroid is None and lane_idx not in centroids and frame_index == 100:
            init_x = int(0.1 * frame_width)
            init_y = int((y_top + y_bottom) / 2)
            centroids[lane_idx] = (init_x, init_y)
        elif new_centroid is not None:
            centroids[lane_idx] = new_centroid

    # Compute speeds (in yards per second) for each lane (if previous centroid available)
    speeds = {}
    for lane_idx in range(5):
        # For lane 5 (index 4) after 7 seconds, set speed to 0.
        if frame_index >= threshold_frame and lane_idx == 4:
            speeds[lane_idx] = 0.0
            continue
        if lane_idx in centroids and lane_idx in prev_centroids:
            (prev_x, prev_y) = prev_centroids[lane_idx]
            (curr_x, curr_y) = centroids[lane_idx]
            distance = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2)
            speeds[lane_idx] = (distance * fps) / pixels_per_yard
        else:
            speeds[lane_idx] = 0.0

    # Draw centroids, labels, and speeds on the frame.
    # For frames after 7 seconds, shift the labels by +1.
    for lane_idx in range(5):
        # Skip drawing lane 5 (swimmer 5) after 7 seconds.
        if frame_index >= threshold_frame and lane_idx == 4:
            continue
        if lane_idx in centroids:
            cx, cy = centroids[lane_idx]
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)
            # If after 7 seconds, add 1 to the label.
            if frame_index >= threshold_frame:
                label = f"Swimmer {lane_idx+1+1}"  # shift label by +1
            else:
                label = f"Swimmer {lane_idx+1}"
            cv2.putText(frame, label, (cx + 10, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
            speed_text = f"{speeds[lane_idx]:.1f} yd/s"
            cv2.putText(frame, speed_text, (cx + 10, cy - 20),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    prev_centroids = centroids.copy()
    out_swimmers.write(frame)
    frame_index += 1

cap_swimmers.release()
out_swimmers.release()
print("Swimmer tracking complete. Output saved as 'tracked_swimmers.mp4'")


Using lane bounds: [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Swimmer tracking complete. Output saved as 'tracked_swimmers.mp4'


In [ ]:
import cv2
import numpy as np

print("Using lane bounds:", lane_bounds)

# Open the tracked swimmers video (which contains the tracking data) and the original video
cap_tracked = cv2.VideoCapture("tracked_swimmers.mp4")
cap_original = cv2.VideoCapture("200free_rotated.mp4")

if not cap_tracked.isOpened() or not cap_original.isOpened():
    print("Error: Could not open one or both videos.")
    exit()

fps = int(cap_original.get(cv2.CAP_PROP_FPS))
frame_width = int(cap_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_final = cv2.VideoWriter("final_tracked_200free.mp4", fourcc, fps, (frame_width, frame_height))

frame_index = 0
# Dictionaries to store centroids for each lane (keys: 0 to 4)
centroids = {}
prev_centroids = {}

# Conversion factor: 61.8 pixels per yard
pixels_per_yard = 61.8

# Dim factor (to make the video a bit darker for overlay clarity)
dim_factor = 0.7

while True:
    ret_tracked, frame_tracked = cap_tracked.read()
    ret_original, frame_original = cap_original.read()

    if not ret_tracked or not ret_original:
        break

    # For the first 100 frames, output the dimmed original frame unmodified.
    if frame_index < 100:
        frame_dimmed = cv2.convertScaleAbs(frame_original, alpha=dim_factor, beta=0)
        out_final.write(frame_dimmed)
        frame_index += 1
        continue

    # Process the tracked video frame to re-extract swimmer centroids.
    gray_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_BGR2GRAY)
    _, swimmer_mask = cv2.threshold(gray_tracked, 200, 255, cv2.THRESH_BINARY)

    # Process each of the 5 lanes (using lane_bounds)
    for lane_idx, (y_top, y_bottom) in enumerate(lane_bounds):
        if lane_idx >= 5:
            break  # only process 5 lanes

        # Extract ROI corresponding to the lane (between the two red lines)
        lane_roi = swimmer_mask[y_top:y_bottom, :]
        contours, _ = cv2.findContours(lane_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:  # filter out noise
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"]) + y_top  # adjust to full-frame coordinates
                    centroids[lane_idx] = (cx, cy)
        # If no blob is detected and it's the first tracking frame, initialize on the left.
        if lane_idx not in centroids and frame_index == 100:
            init_x = int(0.1 * frame_width)
            init_y = int((y_top + y_bottom) / 2)
            centroids[lane_idx] = (init_x, init_y)

    # Compute speed for each lane (in yards per second)
    speeds = {}
    for lane_idx in range(5):
        if lane_idx in centroids and lane_idx in prev_centroids:
            (prev_x, prev_y) = prev_centroids[lane_idx]
            (curr_x, curr_y) = centroids[lane_idx]
            distance = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2)
            speeds[lane_idx] = (distance * fps) / pixels_per_yard
        else:
            speeds[lane_idx] = 0.0

    # Dim the original frame
    frame_dimmed = cv2.convertScaleAbs(frame_original, alpha=dim_factor, beta=0)

    # Overlay centroids, labels, and speeds (all in bold green)
    for lane_idx in range(5):
        if lane_idx in centroids:
            cx, cy = centroids[lane_idx]
            cv2.circle(frame_dimmed, (cx, cy), 5, (0, 255, 0), -1)
            cv2.putText(frame_dimmed, f"Swimmer {lane_idx+1}", (cx + 10, cy),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
            speed_text = f"{speeds[lane_idx]:.1f} yd/s"
            cv2.putText(frame_dimmed, speed_text, (cx + 10, cy - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)

    # Update previous centroids for the next frame
    prev_centroids = centroids.copy()

    out_final.write(frame_dimmed)
    frame_index += 1

cap_tracked.release()
cap_original.release()
out_final.release()
print("Final overlay complete. Output saved as 'final_tracked_200free.mp4'")


Using lane bounds: [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Final overlay complete. Output saved as 'final_tracked_200free.mp4'


In [ ]:
import cv2
import numpy as np

print("Using lane bounds:", lane_bounds)

# Open the tracked swimmers video (which contains the tracking data) and the original video
cap_tracked = cv2.VideoCapture("tracked_swimmers.mp4")
cap_original = cv2.VideoCapture("200free_rotated.mp4")

if not cap_tracked.isOpened() or not cap_original.isOpened():
    print("Error: Could not open one or both videos.")
    exit()

fps = int(cap_original.get(cv2.CAP_PROP_FPS))
frame_width = int(cap_original.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap_original.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_final = cv2.VideoWriter("final_tracked_200free.mp4", fourcc, fps, (frame_width, frame_height))

frame_index = 0
# Dictionaries to store centroids for each lane (keys: 0 to 4)
centroids = {}
prev_centroids = {}

# Conversion factor: 61.8 pixels per yard
pixels_per_yard = 61.8

# Dim factor (to make the original video a bit darker for overlay clarity)
dim_factor = 0.7

# Threshold frame: after 7 seconds, remove lane 5 (index 4) and shift labels by +1.
threshold_frame = int(7 * fps)

while True:
    ret_tracked, frame_tracked = cap_tracked.read()
    ret_original, frame_original = cap_original.read()

    if not ret_tracked or not ret_original:
        break

    # For the first 100 frames, output the dimmed original frame unmodified.
    if frame_index < 100:
        frame_dimmed = cv2.convertScaleAbs(frame_original, alpha=dim_factor, beta=0)
        out_final.write(frame_dimmed)
        frame_index += 1
        continue

    # Process the tracked video frame to re-extract swimmer centroids.
    gray_tracked = cv2.cvtColor(frame_tracked, cv2.COLOR_BGR2GRAY)
    _, swimmer_mask = cv2.threshold(gray_tracked, 200, 255, cv2.THRESH_BINARY)

    # Process each of the 5 lanes (using lane_bounds)
    for lane_idx, (y_top, y_bottom) in enumerate(lane_bounds):
        if lane_idx >= 5:
            break  # only process 5 lanes

        # Extract ROI corresponding to the lane (between the two red lines)
        lane_roi = swimmer_mask[y_top:y_bottom, :]
        contours, _ = cv2.findContours(lane_roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            # Select the largest contour (assumed to be the swimmer)
            largest_contour = max(contours, key=cv2.contourArea)
            if cv2.contourArea(largest_contour) > 30:  # Filter out noise
                M = cv2.moments(largest_contour)
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"]) + y_top  # adjust to full-frame coordinates
                    centroids[lane_idx] = (cx, cy)
        # If no blob is detected and it's the first tracking frame, initialize centroid on the left.
        if lane_idx not in centroids and frame_index == 100:
            init_x = int(0.1 * frame_width)
            init_y = int((y_top + y_bottom) / 2)
            centroids[lane_idx] = (init_x, init_y)

    # Compute speed for each lane (in yards per second)
    speeds = {}
    for lane_idx in range(5):
        if lane_idx in centroids and lane_idx in prev_centroids:
            (prev_x, prev_y) = prev_centroids[lane_idx]
            (curr_x, curr_y) = centroids[lane_idx]
            distance = np.sqrt((curr_x - prev_x) ** 2 + (curr_y - prev_y) ** 2)
            speeds[lane_idx] = (distance * fps) / pixels_per_yard
        else:
            speeds[lane_idx] = 0.0

    # Dim the original frame
    frame_dimmed = cv2.convertScaleAbs(frame_original, alpha=dim_factor, beta=0)

    # Overlay centroids, labels, and speeds on the dimmed original frame.
    if frame_index < threshold_frame:
        # Before 7 seconds, draw all 5 lanes normally.
        for lane_idx in range(5):
            if lane_idx in centroids:
                cx, cy = centroids[lane_idx]
                cv2.circle(frame_dimmed, (cx, cy), 5, (0, 255, 0), -1)
                cv2.putText(frame_dimmed, f"Swimmer {lane_idx+1}", (cx + 10, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
                speed_text = f"{speeds[lane_idx]:.1f} yd/s"
                cv2.putText(frame_dimmed, speed_text, (cx + 10, cy - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
    else:
        # After 7 seconds, do not consider lane 4 (swimmer 5) and shift labels by +1.
        for lane_idx in range(4):  # Only lanes 0 to 3.
            if lane_idx in centroids:
                cx, cy = centroids[lane_idx]
                cv2.circle(frame_dimmed, (cx, cy), 5, (0, 255, 0), -1)
                # Shift label: lane 0 becomes "Swimmer 2", lane 1 becomes "Swimmer 3", etc.
                label = f"Swimmer {lane_idx+1+1}"
                cv2.putText(frame_dimmed, label, (cx + 10, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
                speed_text = f"{speeds[lane_idx]:.1f} yd/s"
                cv2.putText(frame_dimmed, speed_text, (cx + 10, cy - 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
        # Lane 4 (swimmer 5) is not drawn at all.

    # Update previous centroids for the next frame.
    prev_centroids = centroids.copy()

    out_final.write(frame_dimmed)
    frame_index += 1

cap_tracked.release()
cap_original.release()
out_final.release()
print("Final overlay complete. Output saved as 'final_video.mp4'")


Using lane bounds: [(377, 426), (426, 482), (482, 551), (551, 619), (619, 706)]
Final overlay complete. Output saved as 'final_tracked_200free.mp4'
